In [1]:
import pandas as pd
import os
from datetime import datetime, timedelta
import numpy as np
from tqdm import tqdm
from calculateColumnsForModel import calculateColumnsForModel, getAvgOfCollectedData, numberToRoleName

game_ids.xlsx랑 collected_data가 있어야 돌아감.

In [2]:
PARTICIPANT_NUMBER_OF_A_TEAM = 5
participant_ids_by_role = []
DETAIL_PATH = "../data/collected_data/"

In [3]:
def getstats(gameId, number):
    pass

In [4]:
strcolumn_dict = {"tournamentId":"str", "matchId":"str", "gameId":"str", "patch":"str", "esportsTeamId_Blue":"str", "esportsTeamId_Red":"str"}
temp1 = [f"esportsPlayerId_{i}" for i in range(10)]
temp2 = ["str" for i in range(10)]
esportsPlayerId_type_dict = dict(zip(temp1, temp2))
strcolumn_dict.update(esportsPlayerId_type_dict)
game_ids_df = pd.read_excel("../data/game_ids.xlsx", dtype=strcolumn_dict, index_col = 0)
#game_ids_df["startTime(match)"] = game_ids_df["startTime(match)"].apply(lambda x : datetime.datetime.strptime(x[:19], "%Y-%m-%dT%H:%M:%S"))
    # add_esportsTeamId_to_gameId.ipynb에서 이미 형식을 바꿔버려서 위 코드를 넣었을 때 에러가 났던 것. 코드가 커지니까 이제 슬슬 헷갈리네 이러다 실수하는데

game_ids_df.info()


<class 'pandas.core.frame.DataFrame'>
Index: 8021 entries, 0 to 8022
Data columns (total 50 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   startTime(match)         8021 non-null   datetime64[ns]
 1   tournamentId             8021 non-null   object        
 2   blockName                8021 non-null   object        
 3   leagueName               8021 non-null   object        
 4   matchId                  8021 non-null   object        
 5   bestof                   8021 non-null   int64         
 6   blueteam_name            8021 non-null   object        
 7   blueteam_code            8021 non-null   object        
 8   blueteam_win             8021 non-null   int64         
 9   redteam_name             8021 non-null   object        
 10  redteam_code             8021 non-null   object        
 11  redteam_win              8021 non-null   int64         
 12  gameNumberInAMatch       8021 non-null 

In [5]:
#이거 넣어야 됨. id에 널값이 있는 게임은 누군지도 모르니까 걍 제외.
included_all_ids_df = game_ids_df[game_ids_df["esportsPlayerId_0"].notna() & game_ids_df["esportsPlayerId_1"].notna() & game_ids_df["esportsPlayerId_2"].notna() &\
                                game_ids_df["esportsPlayerId_3"].notna() & game_ids_df["esportsPlayerId_4"].notna() & game_ids_df["esportsPlayerId_5"].notna() &\
                                    game_ids_df["esportsPlayerId_6"].notna() & game_ids_df["esportsPlayerId_7"].notna() & game_ids_df["esportsPlayerId_8"].notna() &\
                                        game_ids_df["esportsPlayerId_9"].notna()]
included_all_ids_df.info(verbose=True, show_counts=True)
#결과 줄수 7949 나오면 맞음

<class 'pandas.core.frame.DataFrame'>
Index: 7934 entries, 0 to 8022
Data columns (total 50 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   startTime(match)         7934 non-null   datetime64[ns]
 1   tournamentId             7934 non-null   object        
 2   blockName                7934 non-null   object        
 3   leagueName               7934 non-null   object        
 4   matchId                  7934 non-null   object        
 5   bestof                   7934 non-null   int64         
 6   blueteam_name            7934 non-null   object        
 7   blueteam_code            7934 non-null   object        
 8   blueteam_win             7934 non-null   int64         
 9   redteam_name             7934 non-null   object        
 10  redteam_code             7934 non-null   object        
 11  redteam_win              7934 non-null   int64         
 12  gameNumberInAMatch       7934 non-null 

여기까지는 game_ids에 esportsteamId를 붙이는 코드였음. 집와서 gameId다시읽을때 str타입으로 읽는 코드 추가할 것. strcolumn_dict에 넣으면 됨.

In [6]:
included_all_ids_df_sorted = included_all_ids_df.sort_values(by=["startTime(match)", "gameNumberInAMatch"], ascending=[False, False]).reset_index() # 최근 경기가 위로 오도록 내림차순 정렬
included_all_ids_df_sorted.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7934 entries, 0 to 7933
Data columns (total 51 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   index                    7934 non-null   int64         
 1   startTime(match)         7934 non-null   datetime64[ns]
 2   tournamentId             7934 non-null   object        
 3   blockName                7934 non-null   object        
 4   leagueName               7934 non-null   object        
 5   matchId                  7934 non-null   object        
 6   bestof                   7934 non-null   int64         
 7   blueteam_name            7934 non-null   object        
 8   blueteam_code            7934 non-null   object        
 9   blueteam_win             7934 non-null   int64         
 10  redteam_name             7934 non-null   object        
 11  redteam_code             7934 non-null   object        
 12  redteam_win              7934 non-

In [7]:
included_all_ids_df_sorted.head(5)

index    startTime(match)        tournamentId blockName leagueName  \
0   4020 2024-05-28 10:00:00  112439909522772195    Week 1        LCO   
1   4019 2024-05-28 10:00:00  112439909522772195    Week 1        LCO   
2   4018 2024-05-28 08:00:00  112439909522772195    Week 1        LCO   
3   4017 2024-05-28 08:00:00  112439909522772195    Week 1        LCO   
4   4016 2024-05-27 10:00:00  112439909522772195    Week 1        LCO   

              matchId  bestof       blueteam_name blueteam_code  blueteam_win  \
0  112439909523296569       2  ION Global Esports           ION             2   
1  112439909523296569       2  ION Global Esports           ION             2   
2  112439909523296494       2                Fury           FRY             0   
3  112439909523296494       2                Fury           FRY             0   
4  112439909523296515       2       Antic Esports           ANC             2   

   ...   esportsPlayerId_3   esportsPlayerId_4   esportsPlayerId_5  \
0  ...  111720027901143523  111720051264281664  110462937178590834   
1  ...  111720027901143523  111720051264281664  110462937178590834   
2  ...  108604888219092306  111758035988327173  100160799381721105   
3  ...  108604888219092306  111758035988327173  100160799381721105   
4  ...  102206314367722668  105714276869190870  107634897325128022   

    esportsPlayerId_6   esportsPlayerId_7   esportsPlayerId_8  \
0  111720122152475827  112477352770135560  112477356024384776   
1  111720122152475827  112477352770135560  112477356024384776   
2  104668539132702977  101383792849832093  103643243250697054   
3  104668539132702977  101383792849832093  103643243250697054   
4  107634941727734818  105709436604717285   99566406296347493   

    esportsPlayerId_9 winner_side  esportsTeamId_Blue   esportsTeamId_Red  
0  112477373979171280        Blue  111691749922904280  107437032053655867  
1  112477373979171280        Blue  111691749922904280  107437032053655867  
2  101383792831678607         Red  111691745473532520  109675484017801229  
3  101383792831678607         Red  111691745473532520  109675484017801229  
4  102546029192465275         Red   98767991921462763  111691747754390318  

[5 rows x 51 columns]

In [8]:

test_df = included_all_ids_df_sorted[(included_all_ids_df_sorted["esportsTeamId_Blue"] == "107437032053655867") | (included_all_ids_df_sorted["esportsTeamId_Red"] == "107437032053655867")]
test_df.head(11)

index    startTime(match)        tournamentId  blockName leagueName  \
0     4020 2024-05-28 10:00:00  112439909522772195     Week 1        LCO   
1     4019 2024-05-28 10:00:00  112439909522772195     Week 1        LCO   
720   4085 2024-02-28 07:00:00  111561353913765677  Knockouts        LCO   
721   4084 2024-02-28 07:00:00  111561353913765677  Knockouts        LCO   
726   4083 2024-02-27 09:00:00  111561353913765677  Knockouts        LCO   
727   4082 2024-02-27 09:00:00  111561353913765677  Knockouts        LCO   
835   4072 2024-02-21 07:00:00  111561353913765677     Week 6        LCO   
836   4071 2024-02-21 07:00:00  111561353913765677     Week 6        LCO   
847   4064 2024-02-19 07:00:00  111561353913765677     Week 6        LCO   
848   4063 2024-02-19 07:00:00  111561353913765677     Week 6        LCO   
977   4056 2024-02-13 09:00:00  111561353913765677     Week 5        LCO   

                matchId  bestof       blueteam_name blueteam_code  \
0    112439909523296569       2  ION Global Esports           ION   
1    112439909523296569       2  ION Global Esports           ION   
720  111691792353211336       3       Antic Esports           ANC   
721  111691792353211336       3       Antic Esports           ANC   
726  111691792353211332       3  ION Global Esports           ION   
727  111691792353211332       3  ION Global Esports           ION   
835  111691782055605145       2       Antic Esports           ANC   
836  111691782055605145       2       Antic Esports           ANC   
847  111691782055605109       2         Dire Wolves            DW   
848  111691782055605109       2         Dire Wolves            DW   
977  111691782055605091       2       Kanga Esports           KNG   

     blueteam_win  ...   esportsPlayerId_3   esportsPlayerId_4  \
0               2  ...  111720027901143523  111720051264281664   
1               2  ...  111720027901143523  111720051264281664   
720             2  ...  111720027901143523  111720029171950542   
721             2  ...   98767975937423972  102546029192465275   
726             0  ...  111720048642120764  110456581491929979   
727             0  ...  111720048642120764  110456581491929979   
835             1  ...   98767975937423972  102546029192465275   
836             1  ...   98767975937423972  102546029192465275   
847             0  ...  102206314367722668  105714276869190870   
848             0  ...  102206314367722668  105714276869190870   
977             0  ...  109675733286091757  111720029171950542   

      esportsPlayerId_5   esportsPlayerId_6   esportsPlayerId_7  \
0    110462937178590834  111720122152475827  112477352770135560   
1    110462937178590834  111720122152475827  112477352770135560   
720  107634897325128022   99566406283268035  111720135678641636   
721  110462937178590834  109758314971921663  107635918452357647   
726  110462937178590834  109758314971921663  107635918452357647   
727  110462937178590834  109758314971921663  107635918452357647   
835  110462937178590834  109758314971921663  107635918452357647   
836  110462937178590834  109758314971921663  107635918452357647   
847  110462937178590834  109758314971921663  107635918452357647   
848  110462937178590834  109758314971921663  107635918452357647   
977  100160799381721105  105747868966238958  101383792849832093   

      esportsPlayerId_8   esportsPlayerId_9 winner_side  esportsTeamId_Blue  \
0    112477356024384776  112477373979171280        Blue  111691749922904280   
1    112477356024384776  112477373979171280        Blue  111691749922904280   
720   98767975937423972  102546029192465275         Red  107437032053655867   
721  111720027901143523  111720029171950542        Blue  111691747754390318   
726  111720027901143523  111720029171950542         Red  111691749922904280   
727  111720027901143523  111720029171950542         Red  111691749922904280   
835  111720027901143523  111720029171950542         Red  111691747754390318   
836  111720027901143523  11172002

In [9]:
test_df = included_all_ids_df_sorted[(included_all_ids_df_sorted["esportsPlayerId_0"] == "111720042622049047") | (included_all_ids_df_sorted["esportsPlayerId_5"] == "111720042622049047")]
test_df[["gameId", "esportsPlayerId_0", "esportsPlayerId_5"]].head(11)

gameId   esportsPlayerId_0   esportsPlayerId_5
0     112439909523296571  111720042622049047  110462937178590834
1     112439909523296570  111720042622049047  110462937178590834
726   111691792353211334  111720042622049047  110462937178590834
727   111691792353211333  111720042622049047  110462937178590834
736   111691792353145780  100160799381721105  111720042622049047
737   111691792353145779  100160799381721105  111720042622049047
841   111691782055605156  105714271946959686  111720042622049047
842   111691782055605155  105714271946959686  111720042622049047
971   111691782055605150  107634897325128022  111720042622049047
972   111691782055605149  107634897325128022  111720042622049047
1068  111691782055605129  111720042622049047  101799086969700514

위 테스트 데이터에서 Team 111691749922904280 는 최근 10전 중 3승을 했다.

In [10]:
average_playerstat = getAvgOfCollectedData()    # 한번돌리는시간이 너무 오래결려서 테스트용 임시코드

100%|██████████| 8036/8036 [05:51<00:00, 22.87it/s]


In [11]:
subdata_headtohead = included_all_ids_df_sorted[
    ((included_all_ids_df_sorted["esportsTeamId_Blue"] == "111691749922904280") & 
    (included_all_ids_df_sorted["esportsTeamId_Red"] == "107437032053655867")) |
    ((included_all_ids_df_sorted["esportsTeamId_Blue"] == "107437032053655867") & 
    (included_all_ids_df_sorted["esportsTeamId_Red"] == "111691749922904280"))
].head(11)

In [31]:
# 나중에 현재기준 테이블 만들때는 row안에 데이터를 직접 넣고 가장 겉의 for문 하나 제거하고 뚝딱뚝딱 돌리면 되겠지 ㅎㅎ
# 대신 앞에 last_row_of 저건 읽어와야한다. 반복문에서 하나하나읽지않고 한번 읽고 끝내는 것이 스피드를 위해 핵심.

strcolumn_dict = {"gameId":"str", "esportsTeamId_Blue":"str", "esportsTeamId_Red":"str"}
temp1 = [f"esportsPlayerId_{i}" for i in range(10)]
temp2 = ["str" for i in range(10)]
esportsPlayerId_type_dict = dict(zip(temp1, temp2))
strcolumn_dict.update(esportsPlayerId_type_dict)
last_row_of_collected_datas_df = pd.read_excel("../data/last_row_of_collected_datas.xlsx", dtype = strcolumn_dict)
print("read done")
RANGE_OF_RECENT_GAME = 5   
# RANGE_OF_RECENT_GAME_OF_TEAM = RANGE_OF_RECENT_GAME * 3 # 5개로 하니까 좀 안좋다는 의견이 있어서 15개로 늘려봄 # 그냥 시간기준으로 할까?
#TOO_OLD = datetime(2021, 7, 3)
#average_playerstat = getAvgOfCollectedData()
final_time = included_all_ids_df_sorted.iloc[-1]["startTime(match)"]
pre_make_pivot_table_df = None
for idx, row in tqdm(included_all_ids_df_sorted.iterrows(), total = included_all_ids_df_sorted.shape[0]):
    #if idx + RANGE_OF_RECENT_GAME >= included_all_ids_df.shape[0]: # 거의 끝행에 도달하면 어차피 지난 데이터도 얼마 없기 때문에 스톱한다.
    if row["startTime(match)"] - final_time < timedelta(days=30):  # 너무 오래전꺼는 과거의 데이터가 없어서 힘들다. 2021년 6월 4일자부터만 게임데이터를 가지고 있기 때문에 가장 과거 데이터 1달정도를 자른다.
        break
    else:
        #print(f"현재 row : 블루팀 : {row["esportsTeamId_Blue"]}, 레드팀 : {row["esportsTeamId_Red"]}")
        previous_subdata_df = included_all_ids_df_sorted[idx+1:] # 현재 행의 바로 다음 행부터

        blueteam_wincount = 0       # 블루팀의 승리 횟수
        subdata_blueteam = previous_subdata_df[(previous_subdata_df["esportsTeamId_Blue"] == row["esportsTeamId_Blue"]) | 
                                               (previous_subdata_df["esportsTeamId_Red"] == row["esportsTeamId_Blue"])
                                               ]# 블루 레드팀 어디든 현재 블루팀과 팀이 같은 경기가 있을 떼
        for sub_idx, sub_row in subdata_blueteam.iterrows():
            if row["startTime(match)"] - sub_row["startTime(match)"] > timedelta(days=365): # 최근 1년 경기가 아니면
                break
            else:
                if sub_row["esportsTeamId_Blue"] == row["esportsTeamId_Blue"] and sub_row["winner_side"] == "Blue":
                    blueteam_wincount += 1
                elif sub_row["esportsTeamId_Red"] == row["esportsTeamId_Blue"] and sub_row["winner_side"] == "Red":
                    blueteam_wincount += 1
        blueteam_wincount += 1 # 이건 100%가 나오는걸 피하기 위해 임의로 1승을 추가한 것임.
        blueteam_winrate = blueteam_wincount / (subdata_blueteam.shape[0] + 2)  # + 2의 의미는 임의로 2전을 추가해서 바로위에 wincount+1한거와 함께 1전 1패의 가상데이터를 넣어준거임.

        redteam_wincount = 0        # 레드팀의 승리 횟수
        subdata_redteam = previous_subdata_df[(previous_subdata_df["esportsTeamId_Blue"] == row["esportsTeamId_Red"]) | 
                                              (previous_subdata_df["esportsTeamId_Red"] == row["esportsTeamId_Red"])
                                              ]# 블루 레드팀 어디든 현재 레드팀과 팀이 같은 경기가 있을 떼
        for sub_idx, sub_row in subdata_redteam.iterrows():
            if row["startTime(match)"] - sub_row["startTime(match)"] > timedelta(days=365): # 최근 1년 경기가 아니면
                break
            else:
                if sub_row["esportsTeamId_Blue"] == row["esportsTeamId_Red"] and sub_row["winner_side"] == "Blue":
                    redteam_wincount += 1
                elif sub_row["esportsTeamId_Red"] == row["esportsTeamId_Red"] and sub_row["winner_side"] == "Red":
                    redteam_wincount += 1
        redteam_wincount += 1 # 이건 100%가 나오는걸 피하기 위해 임의로 1승을 추가한 것임.
        redteam_winrate = redteam_wincount / (subdata_redteam.shape[0] + 2)  # + 2의 의미는 임의로 2전을 추가해서 바로위에 wincount+1한거와 함께 1전 1패의 가상데이터를 넣어준거임. 이렇게 100%와 0%의 극단적인 경우를 피해봄
        
        headtohead_wincount = 0     # 같은 팀이 만났을때 블루팀!!의 승리 횟수 (나중에 rate를 구할 때 0~1의 값으로 나올 것임)
        subdata_headtohead = previous_subdata_df[
            ((previous_subdata_df["esportsTeamId_Blue"] == row["esportsTeamId_Blue"]) & 
            (previous_subdata_df["esportsTeamId_Red"] == row["esportsTeamId_Red"])) |
            ((previous_subdata_df["esportsTeamId_Blue"] == row["esportsTeamId_Red"]) & 
            (previous_subdata_df["esportsTeamId_Red"] == row["esportsTeamId_Blue"]))
        ]
        for sub_idx, sub_row in subdata_headtohead.iterrows():
            if row["startTime(match)"] - sub_row["startTime(match)"] > timedelta(days=365): # 최근 1년 경기가 아니면
                break
            else:
                if sub_row["esportsTeamId_Blue"] == row["esportsTeamId_Blue"] and sub_row["winner_side"] == "Blue": # 현재 블루팀과 비교할 블루팀이 같으면서 블루팀이 이겼을 경우
                    headtohead_wincount += 1
                elif sub_row["esportsTeamId_Red"] == row["esportsTeamId_Blue"] and sub_row["winner_side"] == "Red": # 현재 블루팀과 레드팀이 비교할 게임에서 서로 진영을 바꾸었는데 레드팀이 이겼을 경우(결국 같은 팀이 이겼나를 보기 위한 조건이다)
                    headtohead_wincount += 1
        headtohead_wincount += 1
        headtohead_winrate = headtohead_wincount / (subdata_headtohead.shape[0] + 2)
        #print(f"팀 ID {row["esportsTeamId_Blue"]} 와 {row["esportsTeamId_Red"]} 이 붙었을 때 {row["esportsTeamId_Blue"]} 가 이긴 확률은 {headtohead_winrate * 100}% 입니다.")
        
        ##### 아래는 선수별 최근 n경기 스탯을 통해 폼을 뽑아내는 코드 #####
        desired_labels = [f'esportsPlayerId_{j}' for j in range(PARTICIPANT_NUMBER_OF_A_TEAM * 2)]
        for i in range(PARTICIPANT_NUMBER_OF_A_TEAM * 2):   # 선수 10명에 대해
            playerstat_list = []    # 선수별 최근 n경기의 기록을 저장할 곳
            role = i % 5
            subdata_playerstat = previous_subdata_df[(previous_subdata_df[f"esportsPlayerId_{role}"] == row[f"esportsPlayerId_{i}"]) |
                                                     (previous_subdata_df[f"esportsPlayerId_{role + PARTICIPANT_NUMBER_OF_A_TEAM}"] == row[f"esportsPlayerId_{i}"])].head(RANGE_OF_RECENT_GAME)
            number_of_past_games_of_player = subdata_playerstat.shape[0]
            #print(row[f"esportsPlayerId_{i}"])

            for sub_idx, sub_row in subdata_playerstat.iterrows():
                #딕셔너리로 만들어서 playerstat_list에 append하자. 그 다음에 형태를 보자.
                playerstat_dict_of_the_game = {}    # 한 경기에 대한 플레이어 활약상을 넣을 딕셔너리
                game_ids_of_sub_row = sub_row[desired_labels]
                matched_column = game_ids_of_sub_row[game_ids_of_sub_row.values == row[f"esportsPlayerId_{i}"]].index[0]      # ID가 서로 맞는 컬럼이 어디인지(블루팀인지레드팀인지 모르니까)
                matched_column_number = matched_column[matched_column.find("_")+1:]     # 번호 추출
                target_game = calculateColumnsForModel(last_row_of_collected_datas_df[last_row_of_collected_datas_df["gameId"] == sub_row["gameId"]].T.squeeze())
                #print(target_game["gameId"])
                playerstat_dict_of_the_game["kda"] = target_game[f"kda_{matched_column_number}"]
                playerstat_dict_of_the_game["championDamageShare"] = target_game[f"championDamageShare_{matched_column_number}"]
                playerstat_dict_of_the_game["creepScorePerTime"] = target_game[f"creepScorePerTime_{matched_column_number}"]
                playerstat_dict_of_the_game["wardsScorePerTime"] = target_game[f"wardsScorePerTime_{matched_column_number}"]
                playerstat_dict_of_the_game["goldEarnedPerTime"] = target_game[f"goldEarnedPerTime_{matched_column_number}"]
                #print(playerstat_dict_of_the_game)
                playerstat_list.append(playerstat_dict_of_the_game)
                # 넣을거 : kda, championDamageShare, creepScore, wardsScore, goldEarned
            
            #print(number_of_past_games_of_player)
            if number_of_past_games_of_player <= 1:         # 해당 선수의 지난 경기 데이터가 1개 이하이면
                #print("number_of_past_games_of_player <= 1")
                playerstat_dict_of_the_game = {         # 빈 자리에 일반적인 평균을 채워 넣고
                    "kda" : average_playerstat[f"kdaof{numberToRoleName(i)}"],
                    "championDamageShare" : average_playerstat[f"championDamageShareof{numberToRoleName(i)}"],
                    "creepScorePerTime" : average_playerstat[f"creepScorePerTimeof{numberToRoleName(i)}"],
                    "wardsScorePerTime" : average_playerstat[f"wardsScorePerTimeof{numberToRoleName(i)}"],
                    "goldEarnedPerTime" : average_playerstat[f"goldEarnedPerTimeof{numberToRoleName(i)}"]
                }
                for j in range(number_of_past_games_of_player, RANGE_OF_RECENT_GAME):   # 만약 1개면 첫번째만 채워졌을거니 range(1,5)를 해서 4번 반복하는 것이 맞다
                    playerstat_list.append(playerstat_dict_of_the_game)
            else:       # 선수의 지난 데이터가 2개 이상이면, 지난경기가 원한 만큼 다 차도 상관없다 왜냐하면 range에서 리스트를 안 주는 식으로 하면 되니까.
                sum_of_playerstat_dict_of_the_game = {
                    "kda" : 0,
                    "championDamageShare" : 0,
                    "creepScorePerTime" : 0,
                    "wardsScorePerTime" : 0,
                    "goldEarnedPerTime" : 0
                }
                average_of_playerstat_dict_of_the_game = {}
                for playerstat in playerstat_list: # 덧셈과 나눗셈 연산을 써서 시리즈의 평균을 구하자.
                    sum_of_playerstat_dict_of_the_game["kda"] = sum_of_playerstat_dict_of_the_game["kda"] + playerstat["kda"]
                    sum_of_playerstat_dict_of_the_game["championDamageShare"] = sum_of_playerstat_dict_of_the_game["championDamageShare"] + playerstat["championDamageShare"]
                    sum_of_playerstat_dict_of_the_game["creepScorePerTime"] = sum_of_playerstat_dict_of_the_game["creepScorePerTime"] + playerstat["creepScorePerTime"]
                    sum_of_playerstat_dict_of_the_game["wardsScorePerTime"] = sum_of_playerstat_dict_of_the_game["wardsScorePerTime"] + playerstat["wardsScorePerTime"]
                    sum_of_playerstat_dict_of_the_game["goldEarnedPerTime"] = sum_of_playerstat_dict_of_the_game["goldEarnedPerTime"] + playerstat["goldEarnedPerTime"]
                average_of_playerstat_dict_of_the_game["kda"] = sum_of_playerstat_dict_of_the_game["kda"] / number_of_past_games_of_player
                average_of_playerstat_dict_of_the_game["championDamageShare"] = sum_of_playerstat_dict_of_the_game["championDamageShare"] / number_of_past_games_of_player
                average_of_playerstat_dict_of_the_game["creepScorePerTime"] = sum_of_playerstat_dict_of_the_game["creepScorePerTime"] / number_of_past_games_of_player
                average_of_playerstat_dict_of_the_game["wardsScorePerTime"] = sum_of_playerstat_dict_of_the_game["wardsScorePerTime"] / number_of_past_games_of_player
                average_of_playerstat_dict_of_the_game["goldEarnedPerTime"] = sum_of_playerstat_dict_of_the_game["goldEarnedPerTime"] / number_of_past_games_of_player
                for j in range(number_of_past_games_of_player, RANGE_OF_RECENT_GAME):
                    playerstat_list.append(average_of_playerstat_dict_of_the_game)
            if i == 0:  # 첫번째선수니까 데이터프레임을 새로 만들고
                #print("playerstat_list : ", playerstat_list)
                player_form = pd.DataFrame(playerstat_list)
                player_form.reset_index(inplace=True)
                #print("player_form : ", player_form)
                player_form.rename(columns = {"index":"recentGameNumber"}, inplace=True)
            else:       # 아니면 위아래로 이어붙이는 작업
                #print(player_form)
                temp_player_form = pd.DataFrame(playerstat_list)
                temp_player_form.reset_index(inplace=True)
                temp_player_form.rename(columns = {"index":"recentGameNumber"}, inplace=True)
                player_form = pd.concat([player_form, temp_player_form], ignore_index=True)
        #print(player_form)
        # 나온 선수별 폼을 팀별 승률과 이어붙이는 곳
        columns_of_role = []
        for i in range(2):  # 블루팀과 레드팀 2개
            columns_of_role.extend(["Top" for j in range(PARTICIPANT_NUMBER_OF_A_TEAM)])
            columns_of_role.extend(["Jgl" for j in range(PARTICIPANT_NUMBER_OF_A_TEAM)])
            columns_of_role.extend(["Mid" for j in range(PARTICIPANT_NUMBER_OF_A_TEAM)])
            columns_of_role.extend(["Adc" for j in range(PARTICIPANT_NUMBER_OF_A_TEAM)])
            columns_of_role.extend(["Spt" for j in range(PARTICIPANT_NUMBER_OF_A_TEAM)])
        
        each_team_winrate_extended = [blueteam_winrate for i in range(PARTICIPANT_NUMBER_OF_A_TEAM * 5)] # 역할군 갯수가 5개, 총 25개만큼 복제
        each_team_winrate_extended.extend([redteam_winrate for i in range(PARTICIPANT_NUMBER_OF_A_TEAM * 5)])
        columns_of_each_team_winrate_extended = ["Blue" for i in range(PARTICIPANT_NUMBER_OF_A_TEAM * 5)]
        columns_of_each_team_winrate_extended.extend(["Red" for i in range(PARTICIPANT_NUMBER_OF_A_TEAM * 5)])
        headtohead_winrate_extended = [headtohead_winrate for i in range(PARTICIPANT_NUMBER_OF_A_TEAM * 5 * 2)]
        winnerside_extended = [row["winner_side"] for i in range(PARTICIPANT_NUMBER_OF_A_TEAM * 5 * 2)]
        gameid_extended = [row["gameId"] for i in range(PARTICIPANT_NUMBER_OF_A_TEAM * 5 * 2)]
        each_team_winrate_dict = {                                  # 50개 행으로 만들 예정
            "gameId" : gameid_extended,
            "HeadtoHeadWinrate" : headtohead_winrate_extended,
            "winnerSide" : winnerside_extended,                     # 라벨, !!!! 제 일 중 요 !!!!
            "Side" : columns_of_each_team_winrate_extended,         # "Blue" 또는 "Red", 피벗테이블의 컬럼으로 만들 예정
            "TeamWinrate" : each_team_winrate_extended,             # Side에 따라 다른 값
            "Role" : columns_of_role
        }
        dataset_of_a_game_df = pd.DataFrame(each_team_winrate_dict)
        dataset_of_a_game_df = pd.concat([dataset_of_a_game_df, player_form], axis=1, ignore_index=True)
        dataset_of_a_game_df.columns = (["gameId", "HeadtoHeadWinrate", "winner", "Side", "TeamWinrate", "Role", "recentGameNumber", "kda", "championDamageShare", "creepScorePerTime", "wardsScorePerTime", "goldEarnedPerTime"])
        #print(dataset_of_a_game_df)
        if pre_make_pivot_table_df is None:
            pre_make_pivot_table_df = dataset_of_a_game_df
        else:
            pre_make_pivot_table_df = pd.concat([pre_make_pivot_table_df, dataset_of_a_game_df], ignore_index=True)
    # 인덱스는 for문 밖에서 독립적으로 만들어서 잇자, 왜냐하면 현재 기준 테이블 만들 때 코드 같이쓰게.
    # 그럴라그랬는데 귀찮다... 일단 인덱스 만들어서 확인하자
pre_make_pivot_table_df.to_excel("pre_make_pivot_table.xlsx", index=False)    


read done


  0%|          | 6/7934 [00:10<3:43:45,  1.69s/it]


KeyboardInterrupt: 

In [ ]:
"""
i = 0
for i in range(PARTICIPANT_NUMBER_OF_A_TEAM):       # 탑 정글 미드 원딜 서폿 이렇게..
    game_ids_with_playerId_blue = game_ids_df_sorted[pd.isna(game_ids_df_sorted[f"esportsPlayerId_{i}"]) == False]                                  # 블루팀(0~5)에서 서머너ID가 있는 것만
    game_ids_with_playerId_red = game_ids_df_sorted[pd.isna(game_ids_df_sorted[f"esportsPlayerId_{i + PARTICIPANT_NUMBER_OF_A_TEAM}"]) == False]    # 레드팀(5~9)에서 서머너ID가 있는 것만
    participant_ids_by_role.append(list(set(pd.concat([game_ids_with_playerId_blue[f"esportsPlayerId_{i}"],game_ids_with_playerId_red[f"esportsPlayerId_{i + PARTICIPANT_NUMBER_OF_A_TEAM}"]], ignore_index=True))))

print([len(i) for i in participant_ids_by_role])
print(participant_ids_by_role[0])
"""


[346, 370, 368, 355, 394]
['103743600803050303', '101428372795944373', '110528442384858640', '107705430994631113', '105785814323282335', '99921042942435093', '111720165230650519', '101422616382597838', '99566408210057665', '109828500641953854', '112477387590015467', '99566407763820708', '106350736734730852', '105548714234532655', '100725844996918206', '103495716756708191', '108547753236964530', '107492129389636107', '98767991808793901', '101796953888872633', '111720042622049047', '99921042943959644', '105504919708874561', '107635905118503535', '103536968829352943', '104573210012169189', '107559255871511679', '107614620822269031', '101388913286565299', '100438528849437575', '104843445932165305', '99566408342799240', '105501715923396261', '103524818068142935', '99566406287983305', '98767975915355341', '107376143623858213', '99322214668103078', '100285330144336367', '107715200621484403', '100205572901889771', '103495716693930864', '105714276869190870', '109691839332751409', '1013889132918

아 이제 각 포지션별 플레이어ID 리스트 뽑았고...

In [ ]:
"""
file_list_in_detail = os.listdir(DETAIL_PATH)
for i, participant_ids in enumerate(participant_ids_by_role):     # 탑 정글 미드 원딜 서폿에 대해
    for participant_id in tqdm(participant_ids, total = len(participant_ids)):  # 모든 플레이어들에 대해
        filtered_df = game_ids_df_sorted[game_ids_df_sorted[f"esportsPlayerId_{i}"] == participant_id | \
            game_ids_df_sorted[f"esportsPlayerId_{i + PARTICIPANT_NUMBER_OF_A_TEAM}"] == participant_id].head(5)    # 블루팀 안에 있거나 레드팀 안에 있거나 상관없이 제일 최근 경기 5개를 가져온다.
        for j, row in filtered_df.iterrows():       # 와 3중 반복문 이게 맞나...?
            if row["gameId"] + ".xlsx" in file_list_in_detail:     # invalid말고 온전히 남아 있으면
                
                detail_df = pd.read_excel(DETAIL_PATH + row["gameId"] + ".xlsx")   # 데이터를 읽어와서
                #getstats(detail_df)
"""     

 17%|█▋        | 58/346 [04:12<20:51,  4.35s/it]


KeyboardInterrupt: 

이제 각 파일 들어가서 원하는 자료 뽑는 함수 만들어서 저기다 넣으면 됨...
getstats에는 gameId랑 participant_id가 몇번인지 그 번호를 넘길거고 그러기위해서는 몇번인지 찾아놔야한다